In [1]:
# MNIST是一个入门级的计算机视觉数据集，它包含各种手写数字图片。
# MNIST数据集的官网是Yann LeCun's website。在这里，我们提供了一份python源代码用于自动下载和安装这个数据集。
# 参考文献： 
# （1）http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_beginners.html
# （2）https://www.tensorflow.org/get_started/get_started

In [ ]:
# 数据集被分成两部分：60000行的训练数据集（mnist.train）和10000行的测试数据集（mnist.test）
# 每一张图片包含28X28个像素点。
# 长度是 28x28 = 784。如何展开这个数组（数字间的顺序）不重要，只要保持各个图片采用相同的方式展开。
# 从这个角度来看，MNIST数据集的图片就是在784维向量空间里面的点, 并且拥有比较复杂的结构 (提醒: 此类数据的可视化是计算密集型的)。
# 展平图片的数字数组会丢失图片的二维结构信息。这显然是不理想的，最优秀的计算机视觉方法会挖掘并利用这些结构信息，我们会在后续教程中介绍。
# 但是在这个教程中我们忽略这些结构，所介绍的简单数学模型，softmax回归(softmax regression)，不会利用这些结构信息。

# 因此，在MNIST训练数据集中，mnist.train.images 是一个形状为 [60000, 784] 的张量，第一个维度数字用来索引图片，第二个维度数字用来索引每张图片中的像素点。
# 在此张量里的每一个元素，都表示某张图片里的某个像素的强度值，值介于0和1之间。

In [5]:
# MNIST数据集的标签是介于0到9的数字，用来描述给定图片里表示的数字。
# 为了用于这个教程，我们使标签数据是"one-hot vectors"。 一个one-hot向量除了某一位的数字是1以外其余各维度数字都是0。
# 所以在此教程中，数字n将表示成一个只有在第n维度（从0开始）数字为1的10维向量。比如，标签0将表示成([1,0,0,0,0,0,0,0,0,0,0])。
# 因此， mnist.train.labels 是一个 [60000, 10] 的数字矩阵。

## 获取数据集

In [18]:
"""Functions for downloading and reading MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [22]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


## 1. Softmax回归介绍

In [3]:
# softmax回归（softmax regression）模型的经典案例。
# softmax模型可以用来给不同的对象分配概率。即使在之后，我们训练更加精细的模型时，最后一步也需要用softmax来分配概率。

## 2. 定义回归模型

In [ ]:
# 为了用python实现高效的数值计算，我们通常会使用函数库，比如NumPy，会把类似矩阵乘法这样的复杂运算使用其他外部语言实现。
# 不幸的是，从外部计算切换回Python的每一个操作，仍然是一个很大的开销。如果你用GPU来进行外部计算，这样的开销会更大。用分布式的计算方式，也会花费更多的资源用来传输数据。

# TensorFlow也把复杂的计算放在python之外完成，但是为了避免前面说的那些开销，它做了进一步完善。
# Tensorflow不单独地运行单一的复杂计算，而是让我们可以先用图描述一系列可交互的计算操作，然后全部一起在Python之外运行。
#（这样类似的运行方式，可以在不少的机器学习库中看到。）

In [23]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
sess = tf.InteractiveSession(config=config)

In [24]:
# x不是一个特定的值，而是一个占位符placeholder，TensorFlow运行计算时输入这个值。
# 我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。
# 我们用2维的浮点数张量来表示这些图，这个张量的形状是[None，784 ]。（这里的None表示此张量的第一个维度可以是任何长度的。）
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [25]:
# 一行代码来定义我们的模型
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [26]:
# 训练我们的模型，我们首先需要定义一个指标来评估这个模型是好的。交叉熵是用来衡量我们的预测用于描述真相的低效性。

In [27]:
# 为了计算交叉熵，我们首先需要添加一个新的占位符用于输入正确值
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [28]:
# TensorFlow拥有一张描述你各个计算单元的图，它可以自动地使用反向传播算法(backpropagation algorithm)来有效地确定你的变量是如何影响你想要最小化的那个成本值的。
# 然后，TensorFlow会用你选择的优化算法来不断地修改变量以降低成本。
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## 3.训练模型

In [29]:
# 添加一个操作来初始化我们创建的变量
init = tf.global_variables_initializer()
# 在一个Session里面启动我们的模型，并且初始化变量
sess = tf.Session()
sess.run(init)

In [31]:
## 迭代1000次， 每次随机选择100个样本进行小批量训练
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 4.评估模型

In [32]:
# tf.argmax 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值。
# 由于标签向量是由0,1组成，因此最大值1所在的索引位置就是类别标签，比如tf.argmax(y,1)返回的是模型对于任一输入x预测到的标签值，而 tf.argmax(y_,1) 代表正确的标签。
# 我们可以用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)。

In [33]:
# 这行代码会给我们一组布尔值。为了确定正确预测项的比例，我们可以把布尔值转换成浮点数，然后取平均值。
# 例如，[True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# 计算所学习到的模型在测试数据集上面的正确率。
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9123


In [42]:
sess.close()

In [34]:
# 事实上，这个结果是很差的。这是因为我们仅仅使用了一个非常简单的模型。不过，做一些小小的改进，我们就可以得到97％的正确率。
# 最好的模型甚至可以获得超过99.7％的准确率！

In [35]:
# 关于各种模型的性能对比列表
# http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

## 模型改进

In [36]:
# 前面只使用了一个网络隐藏层，没有使用激活函数，只是在输出层使用了softmaxt变换
# 因此，模型的中间部分相当于是线性的。
# 此次，我们考虑使用激活函数relu
# 并且使用正态分布初始化参数
# 使用adam优化方法代替sgd

In [38]:
print('training data shape ', mnist.train.images.shape)
print('testing data shape ', mnist.test.images.shape)
print('training label shape ', mnist.train.labels.shape)

training data shape  (55000, 784)
testing data shape  (10000, 784)
training label shape  (55000, 10)


### 1. 构建网络

In [43]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = False
sess = tf.InteractiveSession(config=config)

In [44]:
# 输入[-1,28*28], FC1 有 1024 个neurons， FC2 有 10 个neurons。这么简单的一个全连接网络，结果测试准确率达到了 0.98。
# 权值初始化
def weight_variable(shape):
    # 用正态分布来初始化权值
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # 本例中用relu激活函数，所以用一个很小的正偏置较好
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


# input_layer
X_ = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# FC1
W_fc1 = weight_variable([784, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(X_, W_fc1) + b_fc1)

# FC2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_pre = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

### 2.训练和评估

In [45]:
# 1.损失函数：cross_entropy
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_pre))
# 2.优化函数：AdamOptimizer, 优化速度要比 GradientOptimizer 快很多
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 3.预测结果评估
#　预测值中最大值（１）即分类结果，是否等于原始标签中的（１）的位置。argmax()取最大值所在的下标
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_, 1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 开始运行
sess.run(tf.global_variables_initializer())
# 这大概迭代了不到 10 个 epoch， 训练准确率已经达到了0.98
for step in range(5000):
    X_batch, y_batch = mnist.train.next_batch(batch_size=100)
    train_step.run(feed_dict={X_: X_batch, y_: y_batch})
    if (step+1) % 200 == 0:
        train_accuracy = accuracy.eval(feed_dict={X_: mnist.train.images, y_: mnist.train.labels})
        print("step %d, training acc %g" % (step+1, train_accuracy))
    if (step+1) % 1000 == 0:
        test_accuracy = accuracy.eval(feed_dict={X_: mnist.test.images, y_: mnist.test.labels})
        print("= " * 10, "step %d, testing acc %g" % (step+1, test_accuracy))

step 200, training acc 0.947382
step 400, training acc 0.964109
step 600, training acc 0.972727
step 800, training acc 0.980164
step 1000, training acc 0.983182
= = = = = = = = = =  step 1000, testing acc 0.9732
step 1200, training acc 0.986964
step 1400, training acc 0.990345
step 1600, training acc 0.989145
step 1800, training acc 0.988745
step 2000, training acc 0.993618
= = = = = = = = = =  step 2000, testing acc 0.9765
step 2200, training acc 0.994364
step 2400, training acc 0.994473
step 2600, training acc 0.996345
step 2800, training acc 0.994073
step 3000, training acc 0.997782
= = = = = = = = = =  step 3000, testing acc 0.9798
step 3200, training acc 0.9984
step 3400, training acc 0.996909
step 3600, training acc 0.998673
step 3800, training acc 0.998364
step 4000, training acc 0.998018
= = = = = = = = = =  step 4000, testing acc 0.9793
step 4200, training acc 0.996618
step 4400, training acc 0.995945
step 4600, training acc 0.997182
step 4800, training acc 0.9992
step 5000, t